In [21]:
import random
import collections
import queue
import argparse
import time

DEFAULT_NUMBER_OF_CUSTOMERS = 5
DEFAULT_END_TIME = 250

DEPARTURE_INTERVAL = 5

Event = collections.namedtuple('Event', 'time proc action')

class Simulator:

    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)

    def run(self, end_time):
        """Schedule and display events until time is up"""
        for _, proc in sorted(self.procs.items()):  
            first_event = next(proc) 
            self.events.put(first_event)
        sim_time = 0  
        while sim_time < end_time:
            if self.events.empty(): 
                print('*** end of events ***')
                break

            current_event = self.events.get()
            sim_time, proc_id, previous_action = current_event 
            print('customer:', proc_id, proc_id * '   ', current_event) 
            active_proc = self.procs[proc_id]
            next_time = sim_time + compute_duration(previous_action)
            try:
                next_event = active_proc.send(next_time)
            except StopIteration:
                del self.procs[proc_id] 
            else:
                self.events.put(next_event)
        else: 
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))


def compute_duration(previous_action):
    if previous_action == 'entered the queue':
        # new state is prowling
        interval = 10
    elif previous_action in ['shop service', 'move next']:
        # new state is trip
        interval = 1
    elif previous_action == 'going home queue too long':
        interval = 0
    else:
        raise ValueError('Unknown previous_action: %s' % previous_action)
    return interval

class Customer():

    queue_size = 0

    def customer_process(self, ident, queue_pos, start_time=0): 
        time = yield Event(start_time, ident, 'entered the queue')  
        queue_pos = self.queue_size
        self.queue_size += 1
        if (queue_pos <= 3):
            for i in range(queue_pos):
                time = yield Event(time, ident, 'move next')
                queue_pos -= 1
            if (queue_pos == 0):
                time = yield Event(time, ident, 'shop service')
        else:
            yield Event(time, ident, 'going home queue too long')  

    def main(self, end_time=DEFAULT_END_TIME, num_customers=DEFAULT_NUMBER_OF_CUSTOMERS, seed=None):
        if seed is not None:
            random.seed(seed)
        customers = {i: self.customer_process(i, self.queue_size, i*DEPARTURE_INTERVAL) for i in range(num_customers)}
        sim = Simulator(customers)
        sim.run(end_time)

customer = Customer()
customer.main()

customer: 0  Event(time=0, proc=0, action='entered the queue')
customer: 1     Event(time=5, proc=1, action='entered the queue')
customer: 0  Event(time=10, proc=0, action='shop service')
customer: 2        Event(time=10, proc=2, action='entered the queue')
customer: 1     Event(time=15, proc=1, action='move next')
customer: 3           Event(time=15, proc=3, action='entered the queue')
customer: 1     Event(time=16, proc=1, action='shop service')
customer: 2        Event(time=20, proc=2, action='move next')
customer: 4              Event(time=20, proc=4, action='entered the queue')
customer: 2        Event(time=21, proc=2, action='move next')
customer: 2        Event(time=22, proc=2, action='shop service')
customer: 3           Event(time=25, proc=3, action='move next')
customer: 3           Event(time=26, proc=3, action='move next')
customer: 3           Event(time=27, proc=3, action='move next')
customer: 3           Event(time=28, proc=3, action='shop service')
customer: 4         